In [1]:
import pandas as pd
import numpy as np
import numba as nba
from sklearn import preprocessing as prep
import gc

In [2]:
@nba.vectorize(nopython=True)
def sigmoid(x):
    # x = np.float32(x)
    return 1 / (1+np.exp(-x))

In [3]:
house_prices_df = pd.read_csv("housepricedata.csv")
house_prices_arr = house_prices_df.values

expected_outputs = house_prices_arr[:,-1]

all_input_neurons_unnorm = house_prices_arr[:,:-1]
all_input_neurons = prep.MinMaxScaler().fit_transform(all_input_neurons_unnorm)
all_input_neurons = all_input_neurons

nr_of_input_samples = all_input_neurons.shape[0]
nr_of_neurons = all_input_neurons.shape[1]
nr_of_neurons, nr_of_input_samples
# del house_prices_df, house_prices_arr, all_input_neurons_unnorm
# gc.collect()

(10, 1460)

In [4]:
# arbitrary choice of 6 hidden neurons
NR_OF_HIDDEN_NEURS = 6

In [5]:
# input -> hidden neurons 1
weights1 = np.random.random((nr_of_neurons, NR_OF_HIDDEN_NEURS))
biases1 = np.random.random(NR_OF_HIDDEN_NEURS)
weights1.shape, biases1.shape

((10, 6), (6,))

In [6]:
# hidden neurons 1 -> hidden neurons 2
weights2 = np.random.random((NR_OF_HIDDEN_NEURS, NR_OF_HIDDEN_NEURS))
biases2 = np.random.random(NR_OF_HIDDEN_NEURS)
weights2.shape, biases2.shape

((6, 6), (6,))

In [7]:
# hidden neurons 2 -> out
weights3 = np.random.random((NR_OF_HIDDEN_NEURS, 2))
biases3 = np.random.random(2)
weights3.shape, biases3.shape

((6, 2), (2,))

# testing ground

In [8]:
def think(input, weights, biases):
    return sigmoid(np.dot(input, weights) + biases)

In [9]:
hidden_1 = think(all_input_neurons, weights1, biases1)
hidden_2 = think(hidden_1, weights2, biases2)
outputs_tuples = think(hidden_2, weights3, biases3)
outputs_tuples.shape, expected_outputs.shape


((1460, 2), (1460,))

In [10]:
@nba.njit
def cost(prob_outputs_tuples, expected_outputs):
    cost = 0.0
    for i in np.arange(len(prob_outputs_tuples)):
            cost += (prob_outputs_tuples[i,0] - expected_outputs[i])**2 + (prob_outputs_tuples[i,1] - expected_outputs[i])**2
    return cost


In [14]:
c = cost(outputs_tuples, expected_outputs)